# Try to Recreate the Sound Module

## Interlude - Get our Build Tools

We have two tools to allow us to build and link ASM68K source files the way it would have been done back in the day. The first is 'rmac', a successor to the popular `madmac` assembler used by Minter:

In [238]:
import os
!rm -rf rmac
!git clone https://github.com/mwenge/rmac.git
os.chdir('rmac')
!make -j5
os.chdir('..')

Cloning into 'rmac'...
remote: Enumerating objects: 2050, done.
remote: Counting objects: 100% (2050/2050), done.
remote: Compressing objects: 100% (390/390), done.
remote: Total 2050 (delta 1641), reused 2050 (delta 1641), pack-reused 0
Receiving objects: 100% (2050/2050), 740.86 KiB | 2.65 MiB/s, done.
Resolving deltas: 100% (1641/1641), done.
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c kwgen.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c 68kgen.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c debug.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c dsp56k.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c dsp56kgen.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c eagen.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -o dsp56kgen dsp56kgen.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -o 68kgen 68kgen.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -o kwgen kwgen.c
gcc -std=c99 -D

listing.c: In function ‘ship_ln’:
listing.c:234:39: warning: ‘%-20s’ directive writing 20 or more bytes into a region of size between 1 and 984 []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wformat-overflow=-Wformat-overflow=]8;;]
  234 |                                 "%-40s%-20s Page %-4d    %s %s        RMAC %01i.%01i.%02i (%s)",
      |                                       ^~~~~
In file included from /usr/include/stdio.h:894,
                 from rmac.h:12,
                 from listing.h:13,
                 from listing.c:16:
/usr/include/x86_64-linux-gnu/bits/stdio2.h:38:10: note: ‘__builtin___sprintf_chk’ output 107 or more bytes (assuming 1128) into a destination of size 1024
   38 |   return __builtin___sprintf_chk (__s, __USE_FORTIFY_LEVEL - 1,
      |          ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
   39 |                                   __glibc_objsize (__s), __fmt,
      |                                   ~~~~~~~~~~~~~~~~~~~~

Next we build `rln`, the successor to `aln` the linker-de-jour for Jaguar:

In [5]:
import os
!rm -rf rln
!git clone http://shamusworld.gotdns.org/git/rln
os.chdir('rln')
!make -j5
os.chdir('..')

Cloning into 'rln'...
remote: Enumerating objects: 216, done.
remote: Counting objects: 100% (216/216), done.
remote: Compressing objects: 100% (216/216), done.
remote: Total 216 (delta 107), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (216/216), 160.16 KiB | 377.00 KiB/s, done.
Resolving deltas: 100% (107/107), done.
/bin/rm -f rln.o
gcc -g -I. -D__GCCUNIX__ -O2 -Wno-format -c rln.c
gcc -g -I. -D__GCCUNIX__ -O2 -Wno-format -o rln rln.o 


## Try recreating using a single source file

In [165]:
!./rmac/rmac -help



 _ __ _ __ ___   __ _  ___ 
| '__| '_ ` _ \ / _` |/ __|
| |  | | | | | | (_| | (__ 
|_|  |_| |_| |_|\__,_|\___|

Renamed Macro Assembler
Copyright (C) 199x Landon Dyer, 2011-2021 Reboot and Friends
V2.1.1 Jun 21 2024 (Linux/OSX)

Usage:
    ./rmac/rmac [options] srcfile

Options:
  -? or -h          Display usage information
  -dsymbol[=value]  Define symbol (with optional value, default=0)
  -e[errorfile]     Send error messages to file, not stdout
  -f[format]        Output object file format
                    a: ALCYON
                    b: BSD (use this for Jaguar)
                    e: ELF
                    p: P56 (use this for DSP56001 only)
                    l: LOD (use this for DSP56001 only)
                    x: com/exe/xex (Atari 800)
                    r: absolute address
  -i[path]          Directory to search for include files
  -l[filename]      Create an output listing file
  -l*[filename]     Create an output listing file without pagination
  -m[cpu]        

In [193]:
#!./rmac/rmac -fa -i moomoo_header.s -o moomoo_header.o
#!./rmac/rmac -s -fb -i moomoo.s -o moomoo.o
!./rmac/rmac -s ~oall -n -ps -i moomoo.s -o moomoo.o


Legacy mode OFF
moomoo.s 114: Warning: unoptimized short branch
moomoo.s 1052: Warning: unoptimized short branch


In [196]:
!dd if=moomoo.o|xxd

28+1 records in
28+1 records out
14458 bytes (14 kB, 14 KiB) copied, 3.4994e-05 s, 413 MB/s
00000000: 601a 0000 21f0 0000 0000 0000 0000 0000  `...!...........
00000010: 1650 0000 0000 0000 0000 0000 4ef9 0000  .P..........N...
00000020: 0174 4ef9 0000 0218 4ef9 0000 0190 4ef9  .tN.....N.....N.
00000030: 0000 059c 4ef9 0000 05c0 4ef9 0000 032a  ....N.....N....*
00000040: 4ef9 0000 0500 4ef9 0000 0552 4ef9 0000  N.....N....RN...
00000050: 056c 4ef9 0000 0584 4ef9 0000 0608 4ef9  .lN.....N.....N.
00000060: 0000 061a 4ef9 0000 061a 4ef9 0000 061a  ....N.....N.....
00000070: 4ef9 0000 06b4 0000 4e75 3f00 08f9 0003  N.......Nu?.....
00000080: 0000 005a 4240 1039 0000 005a 33c0 00f0  ...ZB@.9...Z3...
00000090: 00e0 301f 4e75 436f 7079 7269 6768 7420  ..0.NuCopyright 
000000a0: 2831 3939 3329 2049 6d61 6769 7465 6320  (1993) Imagitec 
000000b0: 4465 7369 676e 2c20 496e 6320 48e7 fffe  Design, Inc H...
000000c0: 23fc 0000 0000 00f1 a114 23fc 0000 0000  #.........#.....
000000d0: 00f1 a100 227c

In [185]:
!./rln/rln -z -u -v -a 4040 xd xd -e moomoo.o -o moomoo_src.abs

      _
 _ __| |_ ___
| '__| | '_  \
| |  | | | | |
|_|  |_|_| |_|

Renamed Linker for Atari Jaguar
Copyright (c) 199x Allan K. Pratt, 2014-2021 Reboot & Friends
V1.7.4 Jun 21 2024 (OSX/Linux)

DoFile() : `moomoo.o' NORMAL
Output file is moomoo_src.abs
Read file moomoo.o
Alcyon object file moomoo.o truncated: Header wants 23116B, file is 14458B
Link aborted.


In [181]:
!diff -y -W 120 <(dd ibs=1 skip=168 if=moomoo_src.abs|xxd) <(dd if=moomoo.bin|xxd)

16+1 records in
16+1 records out
8696 bytes (8.7 kB, 8.5 KiB) copied, 2.7752e-05 s, 313 MB/s
8672+0 records in
16+1 records out
8672 bytes (8.7 kB, 8.5 KiB) copied, 0.00204727 s, 4.2 MB/s
00000000: 4ef9 0000 41ba 4ef9 0000 425e 4ef9 0000  N...A	00000000: 4ef9 0000 41ba 4ef9 0000 425e 4ef9 0000  N...A
00000010: 41d6 4ef9 0000 45d6 4ef9 0000 45fa 4ef9  A.N..   |	00000010: 41d6 4ef9 0000 45e2 4ef9 0000 4606 4ef9  A.N..
00000020: 0000 436c 4ef9 0000 453e 4ef9 0000 458c  ..ClN   |	00000020: 0000 4370 4ef9 0000 44ea 4ef9 0000 4546  ..CpN
00000030: 4ef9 0000 45a6 4ef9 0000 45be 4ef9 0000  N...E   |	00000030: 4ef9 0000 4598 4ef9 0000 45b2 4ef9 0000  N...E
00000040: 4642 4ef9 0000 4654 4ef9 0000 4654 4ef9  FBN..   |	00000040: 45ca 4ef9 0000 464e 4ef9 0000 4660 4ef9  E.N..
00000050: 0000 4654 4ef9 0000 46ee 0000 4e75 3f00  ..FTN   |	00000050: 0000 4660 4ef9 0000 4660 4ef9 0000 4470  ..F`N
00000060: 08f9 0003 0000 40a4 4240 1039 0000 40a4  .....   |	00000060: 0000 4e75 3f00 08f9 0003 0000 40a0 42

## Scratchpad